# Audubon Park Land Encroachment

During my walks in audubon park I have noticed a concerning trend.  It appears at least to me that homeowners are appropriating the public parks intended for all of us to use for the own benefit.  The main vehicle for this appropriation seems to be adding in landscaping which acts as a barrier.  Typically cast-iron or bambo.  This is my attempt to figure whats going on

#### Datasets
* Parcel Map:  https://data.nola.gov/dataset/Parcels/v9q5-fz7t



In [1]:
import geopandas as gpd
#import contextily as cx

import pandas as pd
import numpy as np

import shapely
import shapely.geometry

import geoplot.crs as gcrs
import geoplot as gplt
#import plotly

import folium
#import fiona
import matplotlib.pyplot as plt



In [2]:
useCRS = 3857

In [ ]:
parcelsGpd = gpd.read_file('./datasets/Parcels.geojson', crs='WGS84')

In [ ]:
parcelsGpd = parcelsGpd.to_crs(useCRS)

In [ ]:
parcelsGpd.crs

In [ ]:
#drop public property
parcelsGpd = parcelsGpd[ parcelsGpd['situs_street'] != 'Calhoun' ].copy()  #some kind of public property idk

In [ ]:
parcelsGpd.head(3)

In [ ]:
#hehe get it
audoBounds = [
    (-90.12709, 29.93416),
    (-90.12624, 29.93376),
    (-90.13127, 29.92649),
    (-90.13223, 29.92693),
    
    
]

audoBounds.append(audoBounds[0])

In [ ]:
audoBoundsShape = shapely.geometry.Polygon([shapely.geometry.Point(*pt) for pt in audoBounds])
print(audoBoundsShape)

audoBoundsShape

In [ ]:
audoBoundsShapeGS_wgs84 = gpd.GeoSeries(audoBoundsShape, crs='WGS84')
audoBoundsShapeGS = audoBoundsShapeGS_wgs84.to_crs(useCRS)

In [ ]:
audoHousesMask = parcelsGpd['geometry'].intersects(audoBoundsShapeGS.iloc[0])

(np.count_nonzero(audoHousesMask), parcelsGpd.shape[0])


In [ ]:
audubonCenter = shapely.geometry.box( *parcelsGpd[audoHousesMask].total_bounds ).centroid
list(audubonCenter.coords)[0]

In [ ]:
proj = gcrs.WebMercator()

ax = gplt.webmap(audoBoundsShapeGS.to_crs('WGS84'), projection=proj, zoom=14)
gplt.polyplot(audoBoundsShapeGS.to_crs('WGS84'), ax=ax, color='r', alpha=0.4 )



In [ ]:
m = folium.Map(location=(audubonCenter.y,audubonCenter.x), zoom_start=16)

folium.Choropleth(geo_data=audoBoundsShapeGS, fill_color='red').add_to(m)

print("Target bounds (The area of investigation)")

m

In [ ]:
audubonPark = parcelsGpd[audoHousesMask].loc[parcelsGpd[audoHousesMask]['situs_number'] == '501']
privateProperties = parcelsGpd[audoHousesMask][parcelsGpd[audoHousesMask]['situs_number'] != '501']

In [ ]:
privateProperties['geometry'] = privateProperties['geometry'].map(lambda g:  shapely.ops.unary_union( g.geoms ) )

In [ ]:
proj = gcrs.WebMercator()

fig, ax = plt.subplots(1, figsize=(10,20), subplot_kw={'projection':proj})
                       
                       

gplt.webmap(privateProperties,ax=ax)
gplt.polyplot(audubonPark, ax=ax, color='g', alpha=0.5 )
gplt.polyplot(privateProperties, ax=ax, color='b', alpha=0.4 )


In [ ]:
def getEast(shape):
    coords = list(shape.exterior.coords)
    
    output = (-500,-500)
    
    for coord in coords:
        if coord[0] > output[0]:
            output = coord
            
    return shapely.geometry.Point( output )
    
privateProperties['EastCorner'] = gpd.GeoSeries( privateProperties['geometry'].map(getEast), crs='WGS84' )

In [ ]:
def getSouth(shape):
    coords = list(shape.exterior.coords)
    
    output = (500,500)
    
    for coord in coords:
        if coord[1] < output[1]:
            output = coord
            
    return shapely.geometry.Point( output )
    
privateProperties['SouthCorner'] = gpd.GeoSeries( privateProperties['geometry'].map(getSouth), crs='WGS84' )

In [ ]:
def getPropLine(row):
    return shapely.geometry.LineString([row['SouthCorner'], row['EastCorner']])

privateProperties['PropLine'] = gpd.GeoSeries( privateProperties.apply(getPropLine, axis=1), crs='WGS84')

In [ ]:
print("SouthWestCorners")

def checkCorners(col='EastCorner'):
    m = folium.Map(location=(audubonCenter.y,audubonCenter.x), zoom_start=16)

    folium.Choropleth(geo_data=audubonPark, fill_color='green').add_to(m)
    folium.Choropleth(geo_data=privateProperties['geometry'], fill_color='blue').add_to(m)



    for idx, row in privateProperties.iterrows():
        popup = f"South West Corner of {row['situs_number']} {row['situs_street']} {row['situs_type']}"
        folium.Marker(location=[row[col].y,row[col].x], popup=popup, \
                      icon=folium.Icon(color="red"), \
                      ).add_to(m)


    return m

checkCorners('SouthCorner')

In [ ]:
def getPenaltyBox(row):

    offset = row['PropLine'].parallel_offset( 0.0005)
    
    return shapely.geometry.Polygon( list(row['PropLine'].coords) + list(offset.coords) )


privateProperties['PenalityBox'] = privateProperties.apply(getPenaltyBox, axis=1)

In [ ]:
fig,ax=plt.subplots(figsize=(20,30),subplot_kw={'projection': gplt.WebMercator()})

#ax.set(projection=gplt.WebMercator())

gplt.webmap(privateProperties, ax=ax)

gplt.polyplot(privateProperties, color='blue', edgecolor='blue', ax=ax, alpha=0.3)

np.random.seed(10)

gplt.sankey(privateProperties['PropLine'], ax=ax, linewidth=5,hue=np.random.choice(['r','g', 'b', 'm','o'],privateProperties.shape[0]))

gplt.polyplot(privateProperties['PenalityBox'], ax=ax, edgecolor='r')

